# Model QC Notebook
<img align="right" width="150" height="150" src="https://id.dewberry.com/clients/Templates/DewberryLogo_RGB.png">
### ADCIRC-SWAN Model Setup 


### Description: 

 - What is needed to use this notebook
 - Steps to follow:
      1. Import Libraries & Assign Paths
      2. Assign Tasks (e.g. Check fort.15 for parameters)
      3. Save Completed Document

## 1. Import Libraries & Assign Paths & Vairables

In [ ]:
import adcirc_prod

In [1]:
# Assign Model directory
root = r'/scratch/04952/tmiesse/Dewberry/QC'

input_files = adcirc_prod.GetInputFiles(root)
# returns a dictionary paths to f13,f14,f15
# prints 'Input Files in root:'
# f13 = '/data/path/fort.13'
# e.g. f13: '/data/pat.fort.13'

f13 =input_files.f13
f14 = input_files.f14
f15 = input_files.f1
attrs = f13.attrs 
wave_refr = attrs.wave_refraction_in_swan
all_atts = attrs.GetALL


# Assign Variables 
#attributes = []
#f13, f14, f15 = 'fort.13', 'fort.14', 'fort.15'
#path13  = os.path.join(root, f13)

## 2. Tasks

#### a. Find the parameters used in fort.13

In [2]:
attribute = attributes_fort13(path13)
t_attrib = pd.DataFrame(attribute)
t_attrib.columns = ['Parameter']
t_attrib

,Parameter
0,wave_refraction_in_swan
1,sea_surface_height_above_geoid
2,surface_directional_effective_roughness_length
3,surface_canopy_coefficient
4,mannings_n_at_sea_floor
5,primitive_weighting_in_continuity_equation
6,average_horizontal_eddy_viscosity_in_sea_water...


#### b. Create a table with all the nodal locations

In [3]:
table_v1 = read_fort14(os.path.join(root,f14)) 
table_v1.head()

,node_id,node_x,node_y,value
0,1,-60.0400300000,8.5596800000,6.1570001000
1,2,-60.0402700000,8.6642500000,12.8529997000
2,3,-60.0405100000,8.7688200000,25.3540001000
3,4,-60.0409900000,8.9779600000,46.1559982000
4,5,-60.0414700000,9.1871000000,26.4549999000


#### c. Find the nodes and values for each parameter in fort.13

In [4]:
table_v2 = read_fort13(path13,attribute)
table_v2.head()


 Started finding nodes at 

2018-07-17 08:00:21.632000
===========END========== 

Processing Time : 
0:00:06.158164


,wave,sea,surface_directional,surface_canopy,mannings_n,primitive_weighting,average_horizontal
0,1 0.000000,NaN,15490 0.0106490 0.00182500 0.00100000 0.001000...,16299 0.000000,24262 0.0500000,1 0.0200000,1 4.00000
1,2 0.000000,NaN,15491 0.00421200 0.00109300 0.00100000 0.00100...,21471 0.000000,25236 0.0500000,2 0.00500000,2 4.00000
2,3 0.000000,NaN,16297 0.0511780 0.0586370 0.131352 0.00131000 ...,25239 0.000000,25237 0.0370000,3 0.00500000,3 4.00000
3,4 0.000000,NaN,16298 0.0442180 0.0539520 0.0202930 0.0458700 ...,25245 2.27374e-013,25239 0.100000,4 0.00500000,4 4.00000
4,5 0.000000,NaN,16299 0.109154 0.143936 0.173318 0.123676 0.09...,25255 0.000000,25240 0.0370000,5 0.00500000,5 4.00000


#### d. Split the nodes and the values in the generated table

In [5]:
table_v2['wave_nodes'], table_v2['wave_data'] = table_v2['wave'].str.split(' ', 0).str
table_v2['surface_nodes'], table_v2['e'], table_v2['ese'], table_v2['se'],table_v2['s'], table_v2['sw'], table_v2['wsw'], table_v2['w'], table_v2['wnw'],table_v2['nw'], table_v2['n'], table_v2['ne'], table_v2['ene'] = table_v2['surface_directional'].str.split(' ', 0).str
table_v2['canopy_nodes'], table_v2['canopy_data'] = table_v2['surface_canopy'].str.split(' ', 0).str
table_v2['mannings_nodes'], table_v2['mannings_data'] = table_v2['mannings_n'].str.split(' ', 0).str
table_v2['prim_nodes'], table_v2['prim_data'] = table_v2['primitive_weighting'].str.split(' ', 0).str
table_v2['horiz_nodes'], table_v2['horiz_data'] = table_v2['average_horizontal'].str.split(' ', 0).str
table_v2 = table_v2.drop('wave',1)
table_v2 = table_v2.drop('surface_directional',1)
table_v2 = table_v2.drop('surface_canopy',1)
table_v2 = table_v2.drop('mannings_n',1)
table_v2 = table_v2.drop('primitive_weighting',1)
table_v2 = table_v2.drop('average_horizontal',1)
table_v2.head()

,sea,wave_nodes,wave_data,surface_nodes,e,ese,se,s,sw,wsw,...,ne,ene,canopy_nodes,canopy_data,mannings_nodes,mannings_data,prim_nodes,prim_data,horiz_nodes,horiz_data
0,NaN,1,0.000000,15490,0.0106490,0.00182500,0.00100000,0.00100000,0.00100000,0.00100000,...,0.00104400,0.00242100,16299,0.000000,24262,0.0500000,1,0.0200000,1,4.00000
1,NaN,2,0.000000,15491,0.00421200,0.00109300,0.00100000,0.00100000,0.00100000,0.00100000,...,0.00100000,0.00397500,21471,0.000000,25236,0.0500000,2,0.00500000,2,4.00000
2,NaN,3,0.000000,16297,0.0511780,0.0586370,0.131352,0.00131000,0.00136100,0.00138400,...,0.124346,0.0652540,25239,0.000000,25237,0.0370000,3,0.00500000,3,4.00000
3,NaN,4,0.000000,16298,0.0442180,0.0539520,0.0202930,0.0458700,0.00156900,0.00142100,...,0.0808100,0.0702230,25245,2.27374e-013,25239,0.100000,4,0.00500000,4,4.00000
4,NaN,5,0.000000,16299,0.109154,0.143936,0.173318,0.123676,0.0925650,0.0855250,...,0.265285,0.159219,25255,0.000000,25240,0.0370000,5,0.00500000,5,4.00000


In [6]:
# this table can be made into a csv if that is easier to view
# run this code to write to a csv
table_v2.to_csv(os.path.join(root,'model_qc.csv'))

#### e. Add locations to each node
##### - each attribute will be split out again for this

#### f. Create raster and plot of each attribute

In [11]:
var = table_v2['surface_directional_effective_roughness_length'].iloc[0]

In [135]:
node = var.split(' ')[0]
varss = var.split(' ')[1:]
varss

['0.000000']

In [ ]:
split_vars = lambda x: x.split(' ')[:] 
table_v2['surface_directional_effective_roughness_length'] = table_v2['surface_directional_effective_roughness_length'].apply(split_vars)

In [ ]:
attributes = []
nodesx2, nodesy2, value2, node_id2 = [], [], [], []
loc, location = [], []
nodes = 838458
f13, f14 = 'fort.13', 'fort.14'
node = np.array([], dtype=np.int32)
data = np.array([])

In [ ]:
path13  = os.path.join(root_dir, f13)
attribute = attributes_fort13(path13)
t_attrib = pd.DataFrame(attribute)
t_attrib

In [ ]:
node, data = read_fort13(path13,attribute,attribute[0])


In [ ]:
# Assign date of QC & Name/Info of QC Staff
qc_date = datetime.now()
qc_staff = 'Tyler Miesse'

print('QC Completed on {} by {}'.format(qc_date, qc_staff))

## Export Completed QC Notebook

# END